# Prompt Engineering with OCI Generative AI

_Prompt Engineering_ is the iterative process of crafting specific requests in natural language to instruct large language models (LLMs) to perform a task. Based on the exact language used, the prompt engineer can guide the LLM to provide better or different outputs.

There are different types of prompts:
* **In-context learning**: conditioning an LLM with instructions and or demonstrations of the task it is meant to complete
* **k-shot prompting**: explicitly providing k examples of the intended taks in the prompt

## Setting up the Environment

### Install Python Packages

The execution of this notebook depends on the availability of different Python packages

In [2]:
pip install oci python-dotenv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 37.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 1.8 MB/s eta 0:00:00
  Created wheel for circuitbreaker: filename=circuitbreaker-1.4.0-py3-none-any.whl size=7520 sha256=09496cde3f3b3eb84f573a5c694b3dc94b334ab04e273791e5be7edeb7a633d5
  Stored in directory: /home/codespace/.cache/pip/wheels/3c/be/64/266b6ce2ef1130de5e419f04805acbb2df5a4ab1b91348f25b
Successfully built circuitbreaker
Note: you may need to restart the kernel to use updated packages.


### Import Dependencies & Environment Variables

In [4]:
import os
from dotenv import load_dotenv

import oci

load_dotenv()

False

In [5]:
print(os.getenv('OCI_ENDPOINT'))

None
